## **Association Rule Based Recommender System**

### **İş Problemi**

Aşağıda 3 farklı kullanıcının sepet bilgileri verilmiştir. Bu sepet bilgilerine en uygun ürün önerisini birliktelik kuralı kullanarak yapınız. Ürün önerileri 1 tane ya da 1'den fazla olabilir. Karar kurallarını 2010-2011 Germany müşterileri
üzerinden türetiniz.

* Kullanıcı 1’in sepetinde bulunan ürünün id'si: 21987

* Kullanıcı 2’in sepetinde bulunan ürünün id'si : 23235

* Kullanıcı 3’in sepetinde bulunan ürünün id'si : 22747

### **Veri Seti Hikayesi**

Online Retail II isimli veri seti İngiltere merkezli bir perakende şirketinin 01/12/2009 - 09/12/2011 tarihleri arasındaki online satış
işlemlerini içeriyor. Şirketin ürün kataloğunda hediyelik eşyalar yer almaktadır ve çoğu müşterisinin toptancı olduğu bilgisi
mevcuttur.

#### **Görev 1:** Veriyi Hazırlama

#### **Adım 1:** Online Retail II veri setinden 2010-2011 sheet’ini okutunuz.

In [ ]:
!pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
# çıktının tek bir satırda olmasını sağlar.
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/DSMLBC10/week_7 (10.11.22-16.11.22)/datasets/online_retail_II.xlsx", sheet_name = "Year 2010-2011")

In [ ]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


#### **Adım 2:** StockCode’u POST olan gözlem birimlerini drop ediniz. (POST her faturaya eklenen bedel, ürünü ifade etmemektedir.)

In [ ]:
df1 = df[~df["StockCode"].str.contains("POST", na = False)]
df1

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


#### **Adım 3:** Boş değer içeren gözlem birimlerini drop ediniz.

In [ ]:
df1.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
df1.shape

(405633, 8)

#### **Adım 4:** Invoice içerisinde C bulunan değerleri veri setinden çıkarınız. (C faturanın iptalini ifade etmektedir.)

In [ ]:
df1 = df1[~df1["Invoice"].str.contains("C", na = False)]
df1

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


#### **Adım 5:** Price değeri sıfırdan küçük olan gözlem birimlerini filtreleyiniz.

In [ ]:
df1 = df1[df1["Price"] > 0] 
df1.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,396785.0,13.016349,179.579125,1.000,2.00,6.00,12.00,80995.00
Price,396785.0,3.037677,17.829741,0.001,1.25,1.95,3.75,4161.06
Customer ID,396785.0,15301.463886,1709.852311,12346.000,13975.00,15159.00,16801.00,18287.00


#### **Adım 6:** Price ve Quantity değişkenlerinin aykırı değerlerini inceleyiniz, gerekirse baskılayınız.

In [ ]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit,0)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit,0)
def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        print(f"{col_name}\nLower limit = {low_limit}\nUpper limit = {up_limit}")
        return True
    else:
        print(f"{col_name}\nLower limit = {low_limit}\nUpper limit = {up_limit}")
        return False


In [ ]:
outlier = ["Price", "Quantity"]
for col in outlier:
  replace_with_thresholds(df1, col)


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
df1.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,396785.0,11.854490,25.540170,1.000,2.00,6.00,12.00,298.0
Price,396785.0,2.836271,2.988832,0.001,1.25,1.95,3.75,32.0
Customer ID,396785.0,15301.463886,1709.852311,12346.000,13975.00,15159.00,16801.00,18287.0


#### **Görev 2:** Alman Müşteriler Üzerinden Birliktelik Kuralları Üretme
#### **Adım 1:** Aşağıdaki gibi fatura ürün pivot table’i oluşturacak create_invoice_product_df fonksiyonunu tanımlayınız.

In [ ]:
df1 = df1[df1["Country"]=="Germany"]

In [ ]:
df1.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,2010-12-01 13:04:00,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,2010-12-01 13:04:00,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,2010-12-01 13:04:00,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,2010-12-01 13:04:00,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,2010-12-01 13:04:00,1.95,12662.0,Germany


In [ ]:
create_invoice_product_df = df1.groupby(["Invoice","Description"])["Description"].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)
create_invoice_product_df

Description   50'S CHRISTMAS GIFT BAG LARGE   DOLLY GIRL BEAKER   I LOVE LONDON MINI BACKPACK   RED SPOT GIFT BAG LARGE   SET 2 TEA TOWELS I LOVE LONDON    SPACEBOY BABY GIFT SET  10 COLOUR SPACEBOY PEN  12 COLOURED PARTY BALLOONS  12 IVORY ROSE PEG PLACE SETTINGS  12 MESSAGE CARDS WITH ENVELOPES  12 PENCIL SMALL TUBE WOODLAND  12 PENCILS SMALL TUBE RED RETROSPOT  12 PENCILS SMALL TUBE SKULL  12 PENCILS TALL TUBE POSY  12 PENCILS TALL TUBE RED RETROSPOT  12 PENCILS TALL TUBE SKULLS  12 PENCILS TALL TUBE WOODLAND  12 PINK HEN+CHICKS IN BASKET  12 RED ROSE PEG PLACE SETTINGS  16 PIECE CUTLERY SET PANTRY DESIGN  2 PICTURE BOOK EGGS EASTER BUNNY  2 PICTURE BOOK EGGS EASTER CHICKS  20 DOLLY PEGS RETROSPOT  200 BENDY SKULL STRAWS  200 RED + WHITE BENDY STRAWS  3 DRAWER ANTIQUE WHITE WOOD CABINET  3 HOOK HANGER MAGIC GARDEN  3 HOOK PHOTO SHELF ANTIQUE WHITE  3 PIECE SPACEBOY COOKIE CUTTER SET  3 RAFFIA RIBBONS 50'S CHRISTMAS   3 STRIPEY MICE FELTCRAFT  3 TIER CAKE TIN GREEN AND CREAM  3 TIER CAKE TIN RED AND CREAM  3 TIER SWEETHEART GARDEN SHELF  3 TRADITIONAl BISCUIT CUTTERS  SET  36 DOILIES DOLLY GIRL  36 FOIL HEART CAKE CASES  36 FOIL STAR CAKE CASES   36 PENCILS TUBE RED RETROSPOT  36 PENCILS TUBE SKULLS  36 PENCILS TUBE WOODLAND  3D CHRISTMAS STAMPS STICKERS   3D TRADITIONAL CHRISTMAS STICKERS  3D VINTAGE CHRISTMAS STICKERS   4 LAVENDER BOTANICAL DINNER CANDLES  4 TRADITIONAL SPINNING TOPS  5 HOOK HANGER MAGIC TOADSTOOL  5 HOOK HANGER RED MAGIC TOADSTOOL  50'S CHRISTMAS PAPER GIFT BAG  6 GIFT TAGS 50'S CHRISTMAS   6 GIFT TAGS VINTAGE CHRISTMAS   6 RIBBONS ELEGANT CHRISTMAS   6 RIBBONS EMPIRE    6 RIBBONS RUSTIC CHARM  6 RIBBONS SHIMMERING PINKS   60 CAKE CASES DOLLY GIRL DESIGN  60 CAKE CASES VINTAGE CHRISTMAS  60 TEATIME FAIRY CAKE CASES  72 SWEETHEART FAIRY CAKE CASES  ABC TREASURE BOOK BOX   ACRYLIC GEOMETRIC LAMP  ACRYLIC HANGING JEWEL,PINK  ADULT APRON APPLE DELIGHT  ADVENT CALENDAR GINGHAM SACK  AIRLINE BAG VINTAGE JET SET BROWN  AIRLINE BAG VINTAGE JET SET RED  AIRLINE BAG VINTAGE JET SET WHITE  AIRLINE BAG VINTAGE TOKYO 78  AIRLINE BAG VINTAGE WORLD CHAMPION   ALARM CLOCK BAKELIKE CHOCOLATE  ALARM CLOCK BAKELIKE GREEN  ALARM CLOCK BAKELIKE IVORY  ALARM CLOCK BAKELIKE ORANGE  ALARM CLOCK BAKELIKE PINK  ALARM CLOCK BAKELIKE RED   ALPHABET HEARTS STICKER SHEET  ALPHABET STENCIL CRAFT  ANGEL DECORATION PAINTED ZINC   ANIMAL STICKERS  ANT COPPER LIME BOUDICCA BRACELET  ANT COPPER PINK BOUDICCA BRACELET  ANT COPPER TURQ BOUDICCA BRACELET  ANTIQUE GLASS DRESSING TABLE POT  ANTIQUE GLASS PEDESTAL BOWL  ANTIQUE SILVER T-LIGHT GLASS  APOTHECARY MEASURING JAR  APPLE BATH SPONGE  APRON APPLE DELIGHT  AREA PATROLLED METAL SIGN  ASS FLORAL PRINT MULTI SCREWDRIVER  ASSORTED BOTTLE TOP  MAGNETS   ASSORTED COLOUR BIRD ORNAMENT  ASSORTED COLOUR LIZARD SUCTION HOOK  ASSORTED COLOUR MINI CASES  ASSORTED COLOUR T-LIGHT HOLDER  ASSORTED COLOURS SILK FAN  ASSORTED EASTER DECORATIONS  BELLS  ASSORTED EASTER GIFT TAGS  ASSORTED FLOWER COLOUR "LEIS"  ASSORTED MONKEY SUCTION CUP HOOK  ASSORTED TUTTI FRUTTI BRACELET  ASSORTED TUTTI FRUTTI PEN  ASSTD DESIGN 3D PAPER STICKERS  ASSTD DESIGN RACING CAR PEN  ASSTD FRUIT+FLOWERS FRIDGE MAGNETS  BABUSHKA LIGHTS STRING OF 10  BABY BOOM RIBBONS   BAG 125g SWIRLY MARBLES  BAG 250g SWIRLY MARBLES  BAG 500g SWIRLY MARBLES  BAKING MOULD CHOCOLATE CUPCAKES  BAKING MOULD EASTER EGG WHITE CHOC  BAKING MOULD HEART MILK CHOCOLATE  BAKING MOULD HEART WHITE CHOCOLATE  BAKING MOULD ROSE MILK CHOCOLATE  BAKING MOULD ROSE WHITE CHOCOLATE  BAKING MOULD TOFFEE CUP CHOCOLATE  BAKING SET 9 PIECE RETROSPOT   BAKING SET SPACEBOY DESIGN  BALLOON ART MAKE YOUR OWN FLOWERS  BALLOON PUMP WITH 10 BALLOONS  BALLOON WATER BOMB PACK OF 35  BALLOONS  WRITING SET   BANQUET BIRTHDAY  CARD    BASKET OF TOADSTOOLS  BATH BUILDING BLOCK WORD  BATHROOM METAL SIGN  BEACH HUT SHELF W 3 DRAWERS  BEADED CHANDELIER T-LIGHT HOLDER  BEADED CRYSTAL HEART GREEN LARGE  BELLE JARDINIERE CUSHION COVER  BEWARE OF THE CAT METAL SIGN   BICYCLE PUNCTURE REPA

#### **Adım 2:** Kuralları oluşturacak create_rules fonksiyonunu tanımlayınız ve alman müşteriler için kurallarını bulunuz.

In [ ]:
frequent_itemsets = apriori(create_invoice_product_df,
                            min_support=0.01,
                            use_colnames=True)

frequent_itemsets.sort_values("support", ascending=False)

,support,itemsets
389,0.249443,(ROUND SNACK BOXES SET OF4 WOODLAND )
387,0.160356,(ROUND SNACK BOXES SET OF 4 FRUITS )
311,0.140312,(PLASTERS IN TIN WOODLAND ANIMALS)
362,0.140312,(REGENCY CAKESTAND 3 TIER)
2073,0.133630,"(ROUND SNACK BOXES SET OF 4 FRUITS , ROUND SNA..."
...,...,...
2048,0.011136,"(RETROSPOT PARTY BAG + STICKER SET, WHITE SPOT..."
2047,0.011136,"(RETROSPOT PARTY BAG + STICKER SET, STRAWBERRY..."
2046,0.011136,"(RETROSPOT PARTY BAG + STICKER SET, SPACEBOY L..."
2042,0.011136,"(RETROSPOT PARTY BAG + STICKER SET, SET/3 RED ..."


In [ ]:
rules = association_rules(frequent_itemsets,
                          metric="support",
                          min_threshold=0.01)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(SPACEBOY BEAKER),( DOLLY GIRL BEAKER),0.024499,0.015590,0.013363,0.545455,34.987013,0.012981,2.165702
1,( DOLLY GIRL BEAKER),(SPACEBOY BEAKER),0.015590,0.024499,0.013363,0.857143,34.987013,0.012981,6.828508
2,(LUNCH BAG APPLE DESIGN),(10 COLOUR SPACEBOY PEN),0.062361,0.024499,0.011136,0.178571,7.288961,0.009608,1.187567
3,(10 COLOUR SPACEBOY PEN),(LUNCH BAG APPLE DESIGN),0.024499,0.062361,0.011136,0.454545,7.288961,0.009608,1.719005
4,(SPACEBOY LUNCH BOX ),(10 COLOUR SPACEBOY PEN),0.104677,0.024499,0.013363,0.127660,5.210832,0.010799,1.118257


#### **Görev 2:** Sepet İçerisindeki Ürün Id’leri Verilen Kullanıcılara Ürün Önerisinde Bulunma

#### **Adım 1:** check_id fonksiyonunu kullanarak verilen ürünlerin isimlerini bulunuz.

In [ ]:
def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)
    return product_name
code_names = []
code = [21987, 23235, 22747]
for col in code:
  code_names.append(check_id(df, col))


['PACK OF 6 SKULL PAPER CUPS']
['STORAGE TIN VINTAGE LEAF']
["POPPY'S PLAYHOUSE BATHROOM"]


In [ ]:
names = ['PACK OF 6 SKULL PAPER CUPS', 'STORAGE TIN VINTAGE LEAF', "POPPY'S PLAYHOUSE BATHROOM"]
names

['PACK OF 6 SKULL PAPER CUPS',
 'STORAGE TIN VINTAGE LEAF',
 "POPPY'S PLAYHOUSE BATHROOM"]

#### **Adım 2:** arl_recommender fonksiyonunu kullanarak 3 kullanıcı için ürün önerisinde bulununuz.

In [ ]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]

In [ ]:
for col in names:
  print(arl_recommender(rules, col, 3))

['SET/6 RED SPOTTY PAPER CUPS', 'SET/6 RED SPOTTY PAPER CUPS', 'SET/6 RED SPOTTY PAPER PLATES']
['SET OF 4 KNICK KNACK TINS DOILY ', 'ROUND STORAGE TIN VINTAGE LEAF', 'SET OF TEA COFFEE SUGAR TINS PANTRY']
["POPPY'S PLAYHOUSE KITCHEN", "POPPY'S PLAYHOUSE LIVINGROOM ", "POPPY'S PLAYHOUSE LIVINGROOM "]
